### 수집할 지역
- 망원
- 합정
- 홍대
- 상수
- 연남
- 연희
- 신촌
- 이대
- 서교동

In [5]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../05_data_scraping/.env')
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import time
from konlpy.tag import Mecab
mecab = Mecab()
import re
from datetime import datetime

In [6]:
def string_delete(text) :
    return text.replace("<b>","").replace("</b>", "").replace("&quot;", "")

def parse_date(date_str):
    # 날짜 문자열을 datetime 객체로 변환
    return datetime.strptime(date_str, "%Y%m%d")

def make_df(df_name) :
    for i in df_name :
        for idx in range(0, len(df_name[i])) : 
            df_name[i][idx] = string_delete(df_name[i][idx])
    df_name['postdate'] = df_name['postdate'].apply(parse_date)
    return df_name

In [28]:
# 음식점이랑 카페 뽑으려고 하는 코드
# split기준을 띄어쓰기로 함
def w2v_rc(keyword) :
    client_id = os.getenv('client_id') # 환경 변수에 접근
    client_pw = os.getenv('client_pw') # 환경 변수에 접근

    stopwords = pd.read_csv('https://raw.githubusercontent.com/haram4th/ablearn/main/%ED%95%9C%EA%B5%AD%EC%96%B4%EB%B6%88%EC%9A%A9%EC%96%B4100.txt', header = None)
    split_keyword = keyword.split(' ')
    ex_stop_words = [f'{split_keyword[0]}역']
    for i in ex_stop_words : 
        stopwords = stopwords.drop(stopwords[stopwords[0] == i].index)
    stopwords.reset_index(drop=True, inplace=True)
    
    url = "https://openapi.naver.com/v1/search/blog.json"
    final_data_keyword = pd.DataFrame()
    start = time.time() # 시작
    for i in range(1, 1000) :
        payload = {'query' : keyword, 'display' : 100, 'start' : i, 'sort' : 'sim'}
        headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_pw}
        r = requests.get(url, params = payload, headers = headers)
        if r.status_code == 200 :
            data = r.json()
        else :
            print("Error Code:", r.status_code)
            
        df_keyword_sim = pd.json_normalize(data['items'])
        entire_data_keyword_sim = make_df(df_keyword_sim)
        final_data_keyword = pd.concat([final_data_keyword, entire_data_keyword_sim], ignore_index=True)
        # 형태소 분석 후 후처리
    # def process_title(title, keyword):
    #     # '망리단길'을 임시 토큰으로 변경
    #     split_keyword = keyword.split(' ')
    #     title = title.replace(split_keyword[0], '우주')
    #     # 형태소 분석
    #     words = mecab.pos(re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", title))
    #     # 임시 토큰을 원래의 '망리단길'로 복원
    #     processed_words = [(word[0].replace('우주', keyword), word[1]) for word in words]
    #     return processed_words
        
    # # DataFrame에 적용
    # final_data_keyword['title2'] = final_data_keyword['title'].apply(lambda x : process_title(x, keyword))
    final_data_keyword['title2'] = final_data_keyword['title'].apply(lambda x : re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", x))
    final_data_keyword['title3'] = final_data_keyword['title2'].apply(lambda x : x.split(' '))

    from gensim.models import Word2Vec
    # split_keyword = keyword.split(' ')
    model_keyword_rc = Word2Vec(final_data_keyword['title3'], vector_size=100, window=5, min_count=2, workers=8, sg=0)
    model_keyword_rc_result = model_keyword_rc.wv.most_similar(keyword, topn = 50)
    # 워드투벡터 결과 소수점 4자리로 변경하기
    w2v_rc_result = list((item[0], round(item[1], 4)) for item in model_keyword_rc_result)
    print(f"{time.time()-start:.4f} sec")
    return keyword, w2v_rc_result

In [8]:
mangwon = w2v_rc('망원')
mangwon

/home/user/miniforge3/envs/project/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


276.0947 sec


('망원',
 [('하임', 0.4395),
  ('따식', 0.4203),
  ('더페이머스램', 0.4036),
  ('얘기하기', 0.3997),
  ('갈비파스타', 0.3953),
  ('김밥카페', 0.3946),
  ('역근처', 0.3689),
  ('디저트', 0.3543),
  ('드립커피', 0.3513),
  ('신제품', 0.3491),
  ('하츠코히', 0.3486),
  ('커피가게동경', 0.3435),
  ('쿠리노키제빵', 0.3405),
  ('종', 0.3366),
  ('맛있오', 0.3362),
  ('발표', 0.3362),
  ('방문기', 0.3351),
  ('카페거리', 0.3349),
  ('망원마포구청', 0.3304),
  ('풍경소리가', 0.3283),
  ('잠봉뵈르와', 0.3253),
  ('단렌즈', 0.325),
  ('캐논렌즈', 0.3242),
  ('서울소금집델리', 0.3237),
  ('티노마드', 0.3185),
  ('잡은', 0.3169),
  ('마음에', 0.3142),
  ('드는', 0.3128),
  ('소금방', 0.3113),
  ('줌렌즈', 0.3083),
  ('데이트하기', 0.308),
  ('작별을', 0.3036),
  ('골목집', 0.3008),
  ('남도음식', 0.2984),
  ('퀜치커피', 0.2956),
  ('필링', 0.295),
  ('숨은', 0.2937),
  ('상수', 0.2932),
  ('고하며', 0.2928),
  ('타테이와', 0.2907),
  ('분위기', 0.29),
  ('일본식', 0.2896),
  ('브런치', 0.2877),
  ('단골이', 0.2867),
  ('추천하는', 0.2867),
  ('짜치', 0.2854),
  ('딸기케이크', 0.2852),
  ('감성카페', 0.2851),
  ('', 0.2848),
  ('내돈내산', 0.2838)])

In [10]:
hapjeong = w2v_rc('합정')
hapjeong

288.0541 sec


('합정',
 [('로컬브랜드', 0.3212),
  ('서울시', 0.316),
  ('상권', 0.3134),
  ('미스타교자', 0.3083),
  ('하늘길', 0.305),
  ('데이트', 0.2999),
  ('소개합니다', 0.2945),
  ('가볼만한곳', 0.2906),
  ('엔젤', 0.2864),
  ('본점', 0.2795),
  ('아카데미과학', 0.2793),
  ('합정점을', 0.2733),
  ('체험', 0.2709),
  ('프렌치토스트', 0.27),
  ('해옫', 0.2692),
  ('콘크리트', 0.2661),
  ('홍대', 0.265),
  ('이응이응이응이응', 0.2614),
  ('실내데이트', 0.2591),
  ('인생', 0.2577),
  ('하이볼', 0.2556),
  ('간맥', 0.2511),
  ('애프터눈티', 0.2502),
  ('청첩장모임으로', 0.2497),
  ('코스', 0.2489),
  ('메세나폴리스', 0.2488),
  ('힙한', 0.2474),
  ('익스첼', 0.2433),
  ('합정본점', 0.2424),
  ('데스페라도', 0.2418),
  ('파스타', 0.2415),
  ('신사옥', 0.2406),
  ('트릴로지', 0.2396),
  ('망원', 0.239),
  ('추천', 0.2378),
  ('근처', 0.2375),
  ('그잡채', 0.2375),
  ('야끼니꾸샤브샤브', 0.2365),
  ('수타우동', 0.2363),
  ('바', 0.2349),
  ('합정망원', 0.2346),
  ('레코즈', 0.2338),
  ('추천하는', 0.2334),
  ('메틀', 0.231),
  ('가성비', 0.2297),
  ('칵테일', 0.2292),
  ('미쉐린가이드', 0.229),
  ('철판요리', 0.2286),
  ('빈티지바이로웰', 0.2285),
  ('전문점과', 0.2276)])

In [11]:
hongdae = w2v_rc('홍대')
hongdae

302.9558 sec


('홍대',
 [('해물떡볶이', 0.3574),
  ('치킨난반', 0.3543),
  ('클럽데이', 0.3215),
  ('젤라또', 0.3173),
  ('정시특강', 0.3167),
  ('카쿠시타', 0.3166),
  ('즉석떡볶이', 0.3085),
  ('라이브', 0.307),
  ('모코모코푸딩', 0.3069),
  ('서울', 0.3045),
  ('풍자또간집', 0.2998),
  ('스시이안엔', 0.2984),
  ('망고빙수', 0.2924),
  ('코스요리', 0.2915),
  ('달떡볶이', 0.2914),
  ('마포성산', 0.2906),
  ('학년도', 0.2888),
  ('홍대입구역점', 0.2802),
  ('몽중식', 0.2798),
  ('회', 0.2784),
  ('씨부엉', 0.2775),
  ('빙수', 0.2772),
  ('브런치', 0.2759),
  ('데이트', 0.269),
  ('배달', 0.265),
  ('내돈내산', 0.2608),
  ('발코니가든', 0.2596),
  ('홍대입구역', 0.259),
  ('이국적', 0.2577),
  ('마라카오위', 0.2575),
  ('장소', 0.2544),
  ('이색적', 0.253),
  ('먹거리', 0.2529),
  ('회전초밥', 0.2528),
  ('반티엔야오', 0.249),
  ('홍대분식', 0.2486),
  ('아기자기한', 0.2481),
  ('홍대역', 0.2439),
  ('밤케이크', 0.2414),
  ('같은', 0.2396),
  ('창조의아침', 0.238),
  ('연남동', 0.2369),
  ('카오위', 0.2357),
  ('솔로', 0.2343),
  ('플라자', 0.2336),
  ('홍대입구', 0.2329),
  ('추천', 0.2321),
  ('홍대분위기좋은술집', 0.2312),
  ('총정리', 0.2299),
  ('이색', 0.2296)])

In [12]:
sangsoo = w2v_rc('상수')
sangsoo

292.5271 sec


('상수',
 [('합정망원맛집', 0.3568),
  ('합정', 0.342),
  ('코스', 0.3317),
  ('바', 0.331),
  ('감각적인', 0.3249),
  ('합정상수', 0.3167),
  ('얼반스퀘어내돈내산', 0.3157),
  ('안주', 0.3147),
  ('파는', 0.3051),
  ('소화원', 0.2993),
  ('상수브런치', 0.2983),
  ('동화속', 0.2959),
  ('비오는', 0.2887),
  ('예약은', 0.2865),
  ('베이글정원', 0.2841),
  ('인정한', 0.2839),
  ('감탄했던', 0.2824),
  ('데이트맛집과', 0.2784),
  ('커피도', 0.274),
  ('베니케이크상수역', 0.2721),
  ('임영웅', 0.272),
  ('연예인이', 0.2699),
  ('수상하지만', 0.2695),
  ('테라스에서', 0.2679),
  ('빵도', 0.2679),
  ('혼빵', 0.2675),
  ('맥주가', 0.2651),
  ('빵', 0.2647),
  ('먹잘알', 0.2639),
  ('홍대생', 0.2626),
  ('홍대상수', 0.2611),
  ('보무브런치앤다이닝', 0.2588),
  ('신상', 0.2585),
  ('빵집', 0.2582),
  ('재방문', 0.2576),
  ('토박이의', 0.2554),
  ('추천', 0.2545),
  ('상수역베이커리카페', 0.252),
  ('신서울전', 0.2514),
  ('파스타맛집', 0.2511),
  ('서울', 0.251),
  ('맛있다', 0.2506),
  ('찐하게', 0.2503),
  ('즐기기', 0.2493),
  ('필수', 0.2492),
  ('날', 0.2484),
  ('다양한', 0.2483),
  ('진격의', 0.2481),
  ('양파하학', 0.2463),
  ('자취러의', 0.2448)])

In [13]:
yeonnam = w2v_rc('연남')
yeonnam

267.3033 sec


('연남',
 [('조용한', 0.3486),
  ('만족스러웠던', 0.3316),
  ('감성', 0.3164),
  ('분위기로', 0.3118),
  ('해보는', 0.3047),
  ('찾는', 0.3032),
  ('연남데이트하기', 0.2988),
  ('서울반지공방', 0.29),
  ('휴양지같은', 0.283),
  ('반지만들기', 0.2814),
  ('휘월루', 0.2788),
  ('장소', 0.2787),
  ('터키식', 0.2758),
  ('신상', 0.2749),
  ('탐방', 0.2732),
  ('데이트맛집', 0.2724),
  ('테라스', 0.2687),
  ('아인슈페너맛집', 0.2686),
  ('수제버거', 0.267),
  ('할라피뇨', 0.2635),
  ('오디', 0.2619),
  ('처음', 0.2617),
  ('잘루', 0.2606),
  ('타이', 0.2595),
  ('발견', 0.258),
  ('피쉬', 0.2577),
  ('딸기파르페가', 0.2565),
  ('북촌점', 0.2552),
  ('연말분위기', 0.2543),
  ('주택', 0.2524),
  ('수제피자맛집', 0.2518),
  ('브론시스', 0.2504),
  ('술집연남', 0.2497),
  ('후토마끼가', 0.2493),
  ('얼굴', 0.2483),
  ('사람', 0.2483),
  ('카페', 0.2457),
  ('대화하기', 0.2444),
  ('마들렌', 0.2435),
  ('브라이덜샤워파티룸', 0.2425),
  ('문정이랑', 0.2407),
  ('에그타르트가', 0.2392),
  ('출장', 0.2381),
  ('그릭모모', 0.2359),
  ('루프탑이', 0.2356),
  ('샌드커피', 0.2345),
  ('블루보틀', 0.2334),
  ('홍대입구역', 0.2333),
  ('토끼다이닝', 0.2328),
  ('사진', 0.2316)])

In [26]:
seoul_yeonhui_dong = w2v_rc('서울 연희동')
seoul_yeonhui_dong

270.1497 sec


('서울 연희동',
 [('산도', 0.3119),
  ('핸드드립', 0.2967),
  ('스페셜티커피', 0.2957),
  ('연희동임팩트', 0.2948),
  ('낭낭한', 0.2632),
  ('필터커피', 0.2616),
  ('사고현황', 0.2582),
  ('한적한곳', 0.2543),
  ('후기', 0.2542),
  ('바이인시즌', 0.2486),
  ('가을', 0.2479),
  ('플랫화이트', 0.2449),
  ('가볼만한곳', 0.2433),
  ('코카콜라', 0.2411),
  ('듬뿍', 0.2405),
  ('공사판', 0.2385),
  ('마카롱', 0.2375),
  ('돼지고기', 0.2357),
  ('후르츠', 0.233),
  ('만석집', 0.224),
  ('블루리본서베이', 0.2228),
  ('매뉴팩트커피', 0.2218),
  ('딸기', 0.2218),
  ('커파가게동경', 0.2217),
  ('주차정보', 0.2217),
  ('다녀왔어요', 0.2203),
  ('크리스마스느낌', 0.2193),
  ('고추잡채', 0.219),
  ('냠냠', 0.2183),
  ('최애', 0.2176),
  ('세트', 0.2174),
  ('연세대', 0.2157),
  ('반려동물동반', 0.2146),
  ('이색', 0.2129),
  ('굿', 0.2117),
  ('아인슈페너로', 0.2105),
  ('디저트가', 0.2098),
  ('진심이라면', 0.2097),
  ('그레인', 0.2096),
  ('미술관', 0.2067),
  ('프레스도넛', 0.2059),
  ('공드린애프터눈티', 0.2058),
  ('마우디', 0.2051),
  ('게장정식', 0.2051),
  ('주차위치', 0.2039),
  ('분위기', 0.2036),
  ('티오마카세', 0.2032),
  ('뜨개샵', 0.2011),
  ('주차', 0.2007),
  ('햇살', 0.2005)]

In [30]:
sinchon = w2v_rc('신촌')
sinchon

283.3046 sec


('신촌',
 [('서울신촌', 0.3587),
  ('줄서는집', 0.3497),
  ('또간집', 0.3406),
  ('치즈웨이브', 0.3262),
  ('특수상권', 0.3239),
  ('피자', 0.3211),
  ('사오다', 0.3205),
  ('테이크아웃매장', 0.3178),
  ('내돈내산', 0.3166),
  ('가성비식당', 0.3155),
  ('웨이브', 0.3132),
  ('단체', 0.3124),
  ('왕', 0.3124),
  ('결과물', 0.3112),
  ('돈가스', 0.3108),
  ('시소안경원', 0.3099),
  ('피맥', 0.3084),
  ('카츠업ㅣ혼밥', 0.3064),
  ('멸치견과류김밥제육김밥', 0.3011),
  ('닭발', 0.2958),
  ('샤오롱바오', 0.2939),
  ('댄싱홍콩', 0.2906),
  ('연태고량주를', 0.2897),
  ('치즈가', 0.2873),
  ('이름처럼', 0.2853),
  ('세브란스병원창업', 0.2787),
  ('데이트', 0.2774),
  ('숙성회', 0.2771),
  ('추천', 0.2759),
  ('전망대', 0.2712),
  ('닭꼬치', 0.2646),
  ('라멘', 0.2643),
  ('', 0.2633),
  ('해누리', 0.2626),
  ('돈까스', 0.2611),
  ('홍콩요리', 0.261),
  ('돌솥설렁탕', 0.2608),
  ('두툼한', 0.2591),
  ('연대', 0.2581),
  ('가마마루이라멘', 0.2579),
  ('신상', 0.2577),
  ('몽주방', 0.2546),
  ('방문기', 0.2538),
  ('새송이버섯', 0.2516),
  ('별관', 0.2512),
  ('가능', 0.2509),
  ('맛집', 0.2507),
  ('홍콩', 0.2498),
  ('지엔빙', 0.2489),
  ('일식', 0.2474)])

In [16]:
edae = w2v_rc('이대')
edae

269.6079 sec


('이대',
 [('화로구이', 0.4066),
  ('거리의', 0.3943),
  ('대현매운족발', 0.3833),
  ('명물화로서대문구', 0.3802),
  ('튀김족발', 0.3768),
  ('높이려면', 0.3736),
  ('동네주민인증', 0.3712),
  ('창천동', 0.3651),
  ('신촌', 0.3589),
  ('합격률', 0.3567),
  ('이대역', 0.3564),
  ('예식장', 0.3523),
  ('하면', 0.3431),
  ('찐맛집', 0.3412),
  ('케이터틀', 0.3392),
  ('한술식당든든한', 0.3346),
  ('면접', 0.3323),
  ('기준', 0.3288),
  ('이대케이크', 0.3225),
  ('고깃집', 0.3194),
  ('신촌웨딩홀', 0.3181),
  ('입시', 0.3178),
  ('유소바', 0.3158),
  ('연탄구이', 0.315),
  ('대학교', 0.3135),
  ('대족발이예요', 0.3117),
  ('언제', 0.3102),
  ('수제버거', 0.3074),
  ('추천', 0.3065),
  ('삼촌버거', 0.303),
  ('그루밍커피', 0.3027),
  ('가성비', 0.3013),
  ('시즌엔', 0.3006),
  ('철부지모임공간', 0.2997),
  ('연영과입시연극영화과', 0.2991),
  ('악기연주합창', 0.2979),
  ('이용하는', 0.2975),
  ('리화인와플이대역', 0.2966),
  ('바스크치즈', 0.2962),
  ('신상카페', 0.294),
  ('좋아요', 0.2937),
  ('메이크업룸', 0.2927),
  ('졸업연주', 0.2921),
  ('신촌카페', 0.2912),
  ('구워', 0.2897),
  ('김밥사랑', 0.2896),
  ('필름', 0.2895),
  ('핵심만', 0.2891),
  ('스탠바이키친', 0.2891),
  ('비대칭', 0.

In [17]:
seogyo = w2v_rc('서교동')
seogyo

263.8967 sec


('서교동',
 [('마포홍대', 0.3578),
  ('속', 0.3197),
  ('멧커피', 0.3102),
  ('서교동연탄집ㅣ회식', 0.3088),
  ('다이닝', 0.2786),
  ('합정파인다이닝', 0.2683),
  ('장소로도', 0.2654),
  ('끝판왕', 0.2645),
  ('홍대', 0.2594),
  ('찾기', 0.2592),
  ('대구탕', 0.2586),
  ('하우스', 0.2586),
  ('오리', 0.2583),
  ('대서리', 0.257),
  ('추천할만한', 0.2568),
  ('겸', 0.2536),
  ('시원한', 0.253),
  ('칼국수', 0.253),
  ('줄서는', 0.2453),
  ('서교밥집', 0.245),
  ('메뉴와', 0.2439),
  ('한식술집', 0.239),
  ('가성비있는', 0.2377),
  ('집인', 0.237),
  ('누', 0.2367),
  ('세기적일상', 0.2359),
  ('수상한베이글', 0.2354),
  ('찾는다면', 0.234),
  ('일주일만에', 0.2334),
  ('퓨전', 0.2328),
  ('신뢰할', 0.2313),
  ('런치코스', 0.2307),
  ('집이자', 0.2306),
  ('이티씨펫클럽', 0.2297),
  ('중동', 0.2295),
  ('연가정', 0.2293),
  ('덮밥', 0.2288),
  ('카다멈번', 0.2287),
  ('초역세권위치', 0.2284),
  ('오코노미야끼', 0.2278),
  ('정기진행', 0.2263),
  ('서교동피자', 0.2257),
  ('스프링플레어서점', 0.2253),
  ('비용', 0.2252),
  ('아인슈페너와', 0.2247),
  ('점심메뉴', 0.224),
  ('고기', 0.2234),
  ('우와', 0.2229),
  ('일본식벤또', 0.2228),
  ('로스팅', 0.2213)])